### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Total number of Schools and Students
total_schools = len(school_data_complete.school_name.unique())
total_students = len(school_data_complete['Student ID'].unique())

#Total Budget
total_budget = school_data.budget.sum()

#Average Math and Reading score
avg_math = student_data.math_score.mean()
avg_reading = student_data.reading_score.mean()

#Percentage of students >= 70 on Math and Reading
passing_math = len(student_data[student_data['math_score'] >= 70]) / total_students * 100
passing_reading = len(student_data[student_data['reading_score'] >= 70]) / total_students * 100

#Percentage of students who passed BOTH math AND reading >= 70 (overall)
passing_both = student_data[(student_data['math_score'] >= 70) & (student_data['reading_score'] >= 70)]['Student ID'].count() / total_students * 100

In [3]:
#Create DataFrame
district_summary = pd.DataFrame({
    'Total Schools': [total_schools],
    'Total Students': [total_students],
    'Total Budget': [total_budget],
    'Average Math Score': [avg_math],
    'Average Reading Score': [avg_reading],
    '% Passing Math': [passing_math],
    '% Passing Reading': [passing_reading],
    '% Overall Passing': [passing_both]
})

#Format DataFrame
district_summary = district_summary.style.format({
    'Total Students': "{:,}",
    'Total Budget': "${:,.2f}",
    'Average Math Score': "{:.2f}",
    'Average Reading Score': "{:.2f}",
    '% Passing Math': "{:.2f}",
    '% Passing Reading': "{:.2f}",
    '% Overall Passing': "{:.2f}"
})

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
#Using group by function to group by school to create school summary
grouped_schools = school_data_complete.set_index('school_name').groupby('school_name')

#Students by school
stu_count_schools = grouped_schools['Student ID'].count()

#Determine school type
school_types = school_data.set_index('school_name')['type']

#Determine budget and per student budget
total_school_budget = school_data.set_index('school_name')['budget']
per_stu_budget = total_school_budget / school_data.set_index('school_name')['size']

#Determine avg scores by school
school_avg_math = grouped_schools.math_score.mean()
school_avg_reading = grouped_schools.reading_score.mean()

#Determine percentage passing rates
perc_school_passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count() / stu_count_schools
perc_school_passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count() / stu_count_schools
perc_school_passing_overall = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)].groupby('school_name')['Student ID'].count() / stu_count_schools


In [5]:
#Create DataFrame
school_summary = pd.DataFrame({
    'School Type': school_types,
    'Total Students': stu_count_schools,
    'Total School Budget': total_school_budget,
    'Per Student Budget': per_stu_budget,
    'Average Math Score': school_avg_math,
    'Average Reading Score': school_avg_reading,
    '% Passing Math': perc_school_passing_math,
    '% Passing Reading': perc_school_passing_reading,
    '% Overall Passing': perc_school_passing_overall
})
school_summary.index.name = None

#Format the DF
school_summary_df = school_summary.style.format({
    'Total Students': '{:,}',
    'Total School Budget': '${:,.0f}',
    'Per Student Budget': '${:.0f}',
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2%}',
    '% Passing Reading': '{:.2%}',
    '% Overall Passing': '{:.2%}' 
})

school_summary_df.index.name = None
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [6]:
top_five_schools = school_summary.sort_values('% Overall Passing', ascending=False)

top_five_schools.head().style.format({
    'Total Students': '{:,}',
    'Total School Budget': '${:,.0f}',
    'Per Student Budget': '${:.0f}',
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2%}',
    '% Passing Reading': '{:.2%}',
    '% Overall Passing': '{:.2%}'})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [7]:
top_five_schools = school_summary.sort_values('% Overall Passing')

top_five_schools.head().style.format({
    'Total Students': '{:,}',
    'Total School Budget': '${:,.0f}',
    'Per Student Budget': '${:.0f}',
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2%}',
    '% Passing Reading': '{:.2%}',
    '% Overall Passing': '{:.2%}'})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
#Data to be grouped by grade level and school in variables
ninth_math = school_data_complete[school_data_complete['grade'] == '9th'].groupby('school_name')['math_score'].mean()
tenth_math = school_data_complete[school_data_complete['grade'] == '10th'].groupby('school_name')['math_score'].mean()
eleventh_math = school_data_complete[school_data_complete['grade'] == '11th'].groupby('school_name')['math_score'].mean()
twelfth_math = school_data_complete[school_data_complete['grade'] == '12th'].groupby('school_name')['math_score'].mean()

#Put the variables in a DF
math_by_grade = pd.DataFrame({
    '9th': ninth_math,
    '10th': tenth_math,
    '11th': eleventh_math,
    '12th': twelfth_math,
})

#Cleaner formatting
math_by_grade = math_by_grade.style.format({
    '9th': '{:.2f}',
    '10th': '{:.2f}',
    '11th': '{:.2f}',
    '12th': '{:.2f}',
})

math_by_grade.index.name = None

math_by_grade

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
#Data to be grouped by grade level and school in variables
ninth_reading = school_data_complete[school_data_complete['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
tenth_reading = school_data_complete[school_data_complete['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
eleventh_reading = school_data_complete[school_data_complete['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
twelfth_reading = school_data_complete[school_data_complete['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

#Put the variables in a DF
reading_by_grade = pd.DataFrame({
    '9th': ninth_reading,
    '10th': tenth_reading,
    '11th': eleventh_reading,
    '12th': twelfth_reading,
})

#Cleaner formatting
reading_by_grade = reading_by_grade.style.format({
    '9th': '{:.2f}',
    '10th': '{:.2f}',
    '11th': '{:.2f}',
    '12th': '{:.2f}',
})

reading_by_grade.index.name = None

reading_by_grade

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
#Create the bins and labels
bins = [0, 585, 630, 645, 680]
labels = ['<$585', '$585-630', '$630-645', '$645-680']

#Slice the data and place it into bins with a new column for main df
school_data_complete['Spending Ranges(Per Student)'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], bins, labels=labels)

#Grouped by spending range column
school_data_complete_binned = school_data_complete.groupby('Spending Ranges(Per Student)')

#Variables for calculations
binned_avg_math = school_data_complete_binned['math_score'].mean()
binned_avg_reading = school_data_complete_binned['reading_score'].mean()
binned_perc_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('Spending Ranges(Per Student)')['Student ID'].count()/school_data_complete_binned['Student ID'].count()
binned_perc_reading = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('Spending Ranges(Per Student)')['Student ID'].count()/school_data_complete_binned['Student ID'].count()
binned_perc_overall = (binned_perc_math + binned_perc_reading)/2

#Create a DF for the above variables
scores_by_school_spend = pd.DataFrame({
    'Average Math Score': binned_avg_math,
    'Average Reading Score': binned_avg_reading,
    '% Passing Math': binned_perc_math,
    '% Passing Reading': binned_perc_reading,
    '% Overall Passing': binned_perc_overall,
})

#Formatting
scores_by_school_spend = scores_by_school_spend.style.format({
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2%}',
    '% Passing Reading': '{:.2%}',
    '% Overall Passing': '{:.2%}',
})

scores_by_school_spend


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges(Per Student),,,,,
<$585,83.36,83.96,93.70%,96.69%,95.19%
$585-630,79.98,82.31,79.11%,88.51%,83.81%
$630-645,77.82,81.30,70.62%,82.60%,76.61%
$645-680,77.05,81.01,66.23%,81.11%,73.67%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [11]:
#Create the bins and labels
bins = [0, 1000, 2000, 5000]
labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

#Slice the data and place it into bins with a new column for main df
school_data_complete['School Size'] = pd.cut(school_data_complete['size'], bins, labels=labels)

#Grouped by spending range column
school_data_complete_binned = school_data_complete.groupby('School Size')

#Variables for calculations
binned_avg_math = school_data_complete_binned['math_score'].mean()
binned_avg_reading = school_data_complete_binned['reading_score'].mean()
binned_perc_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('School Size')['Student ID'].count()/school_data_complete_binned['Student ID'].count()
binned_perc_reading = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('School Size')['Student ID'].count()/school_data_complete_binned['Student ID'].count()
binned_perc_overall = (binned_perc_math + binned_perc_reading)/2

#Create a DF for the above variables
scores_by_school_size = pd.DataFrame({
    'Average Math Score': binned_avg_math,
    'Average Reading Score': binned_avg_reading,
    '% Passing Math': binned_perc_math,
    '% Passing Reading': binned_perc_reading,
    '% Overall Passing': binned_perc_overall,
})

#Formatting
scores_by_school_size = scores_by_school_size.style.format({
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2%}',
    '% Passing Reading': '{:.2%}',
    '% Overall Passing': '{:.2%}',
})

scores_by_school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.83,83.97,93.95%,96.04%,95.00%
Medium (1000-2000),83.37,83.87,93.62%,96.77%,95.19%
Large (2000-5000),77.48,81.20,68.65%,82.13%,75.39%


## Scores by School Type

* Perform the same operations as above, based on school type

In [12]:
#Grouped by spending range column
school_data_complete_binned = school_data_complete.groupby('type')

#Variables for calculations
binned_avg_math = school_data_complete_binned['math_score'].mean()
binned_avg_reading = school_data_complete_binned['reading_score'].mean()
binned_perc_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/school_data_complete_binned['Student ID'].count()
binned_perc_reading = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/school_data_complete_binned['Student ID'].count()
binned_perc_overall = (binned_perc_math + binned_perc_reading)/2

#Create a DF for the above variables
scores_by_school_type = pd.DataFrame({
    'Average Math Score': binned_avg_math,
    'Average Reading Score': binned_avg_reading,
    '% Passing Math': binned_perc_math,
    '% Passing Reading': binned_perc_reading,
    '% Overall Passing': binned_perc_overall,
})

#Formatting
scores_by_school_type = scores_by_school_type.style.format({
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2%}',
    '% Passing Reading': '{:.2%}',
    '% Overall Passing': '{:.2%}',
})

scores_by_school_type.index.name = 'Type of School'
scores_by_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Type of School,,,,,
Charter,83.41,83.90,93.70%,96.65%,95.17%
District,76.99,80.96,66.52%,80.91%,73.71%


### Report Summary
Observable Trends

1. Looking at the top 5 and worst 5 performing schools based on overall passing %, it seems that despite their location on the top 5 or worst 5, the trend is that reading scores generally tend to be higher than the math scores for each respective school. Although for the bottom 5 performing scores, math scores suffered much more than the math scores for the top 5 while the reading scores were a little bit closer in range to each other than the drastic range in math scores.

2. Also, based on the top 5 and worst 5 performing schools, it can be seen that the top 5 schools are all charter schools while the bottom 5 are all district schools based on type. This seems interesting enough and by looking at the Scores by School Type right above confirms that Charter schools perform better than District schools as Charters are better in every metric from this grouped by school type chart.
